In [16]:
# Importy
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from typing import Callable

In [17]:
# Pobranie danych
heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
feature_matrix = heart_disease.data.features.dropna()
labels = heart_disease.data.targets.loc[feature_matrix.index]

# Przetworzenie zbioru wartości przewidywanych do wartości binarnych
y_binary = labels.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)                      

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(feature_matrix, columns=['cp', 'restecg', 'slope','ca','thal'])         

# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y_binary, test_size=0.2, random_state=268555)

# Normalizacja cech trenignowych i testowych
scaler = StandardScaler()
x_train_normalised = scaler.fit_transform(x_train)
x_test_normalised = scaler.transform(x_test)

# Konwersja danych do wymaganego formatu
x_train = np.array(x_train).astype(float)
y_train = np.array(y_train).astype(float)

In [18]:
# Funkcje aktywacji i ich pochodne
def relu(x: np.ndarray) -> np.ndarray:
    return np.maximum(0, x)

def relu_derivative(x: np.ndarray) -> np.ndarray:
    return (x > 0).astype(float)

def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x: np.ndarray) -> np.ndarray:
    s = sigmoid(x)
    return s * (1 - s)

In [19]:
# Funkcja kosztu i jej pochodna
def binary_cross_entropy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return -np.mean(y_true * np.log(y_pred + 1e-15) + (1 - y_true) * np.log(1 - y_pred + 1e-15))

def binary_cross_entropy_derivative(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    return (y_pred - y_true) / (y_pred * (1 - y_pred) + 1e-15)

In [20]:
# Klasa pojedynczej warstwy
class SingleLayer:
    def __init__(self,
                 input_layer_size : int,
                 output_layer_size : int,
                 activation : Callable[[np.ndarray, np.ndarray], float],
                 activation_derivative : Callable[[np.ndarray, np.ndarray], float],
                 std_dev : float = 0.01
                 )-> None:
        self.weights : np.ndarray = np.random.randn(input_layer_size, output_layer_size) * std_dev
        self.biases : np.ndarray = np.zeros((1, output_layer_size))
        self.activation : Callable[[np.ndarray, np.ndarray], float] = activation
        self.activation_derivative : Callable[[np.ndarray, np.ndarray], float] = activation_derivative
    
    # oblicza sume ważoną wejść,
    # zapisuje feature_matrix do celow późniejszej propagacji wstecznej,
    # zwraca wartości funkcji aktywacji
    def forward(self, feature_matrix: np.ndarray) -> np.ndarray:
        self.weighted_matrix = feature_matrix @ self.weights + self.biases
        self.cache_x = feature_matrix
        return self.activation(self.weighted_matrix)

    # oblicza deltę, gradient wag i gradient biasów,
    # zwraca iloczyn delta i transponowanych wag, który posłuży jako gradient dla poprzedniej warstwy
    def backward(self, gradient: np.ndarray) -> np.ndarray:
        delta = gradient * self.activation_derivative(self.weighted_matrix)
        self.d_weights = self.cache_x.T @ delta
        self.d_biases = np.sum(delta, axis=0, keepdims=True)
        return delta @ self.weights.T

    # akutalizacja wag
    def update(self, learning_rate: float) -> None:
        self.weights -= learning_rate * self.d_weights
        self.biases -= learning_rate * self.d_biases

In [21]:
# Klasa sieci neuronowej
class NeuralNetwork:
    def __init__(self, layer_dims: list[int], std_dev: float = 0.01) -> None:
        self.layers: list[SingleLayer] = []
        for i in range(len(layer_dims) - 1):
            input_size: int = layer_dims[i]
            output_size: int = layer_dims[i + 1]
            if i < len(layer_dims) - 2:  # warstwy ukryte
                self.layers.append(SingleLayer(input_size, output_size, relu, relu_derivative, std_dev))
            else:  # ostatnia warstwa wyjściowa
                self.layers.append(SingleLayer(input_size, output_size, sigmoid, sigmoid_derivative, std_dev))

    # wykonanie sprzężenia w przód na wszystkich warstwach 
    def forward(self, feature_matrix: np.ndarray) -> np.ndarray:
        for layer in self.layers:
            feature_matrix = layer.forward(feature_matrix)
        return feature_matrix

    # wykonanie propagacji wstecznej na wszystkich warstwach 
    def backward(self, y_pred: np.ndarray, y_true: np.ndarray) -> None:
        gradient: np.ndarray = binary_cross_entropy_derivative(y_true, y_pred)
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)

    # wykonanie aktualizacji wag dla wszsytkich warstw
    def update_weights(self, learning_rate: float) -> None:
        for layer in self.layers:
            layer.update(learning_rate)

    # przeszkolenie sieci przez określoną liczbę epok
    def train(self, feature_matrix: np.ndarray, labels: np.ndarray, epochs = 1000, learning_rate_param = 0.01) -> None:
        global learning_rate
        learning_rate = learning_rate_param
        for epoch in range(epochs):
            y_pred = self.forward(feature_matrix)
            cost = binary_cross_entropy(labels, y_pred)
            self.backward(y_pred, labels)
            self.update_weights(learning_rate)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Cost: {cost}")

    # predykcja wyniku
    def predict(self, X: np.ndarray) -> np.ndarray:
        return (self.forward(X) > 0.5).astype(int)

In [ ]:
layers = [[25, 1],
          [25, 25, 1],
          [25, 5, 1],
          [25, 25, 5, 1],
          [25, 12, 1],
          [25, 8, 4, 1],
          [25, 12, 6, 3, 1],
          [25, 12, 8, 4, 1],
          [25, 12, 8, 4, 2, 1]
          ]

learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

epochs_list = [1000, 5000, 10000]

data_sets = [(x_train, x_test), (x_train_normalised, x_test_normalised)]

std_devs = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

if __name__ == "__main__":
    for layer in layers:
        for single_learning_rate in learning_rates:
            for epoch in epochs_list:
                for data_set in data_sets:
                    for std_dev in std_devs:
                        print(f'%s\t%s\t%s\t%s\t%s\t' % layer, single_learning_rate, epoch, data_set, std_dev)
                        # Tworzenie, trenowanie i testowanie modelu
                        model = NeuralNetwork(layer, std_dev)
                        model.train(data_set[0], y_train)

                        # Sprawdzenie predykcji
                        predictions = model.predict(data_set[1])
                        accuracy = np.mean(predictions == y_test)
                        print(f"Accuracy: {accuracy}")

TypeError: not enough arguments for format string